In [151]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [152]:
import numpy as np
import pandas as pd

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [153]:
type(train)

pandas.core.frame.DataFrame

In [154]:
train.shape

(891, 12)

In [155]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [156]:
# drop   : 필요없는 데이터 걸러내기
# axis=0 : 행 방향으로 동작
# axis=1 : 열 방향으로 동작
train = train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
test = test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [157]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [158]:
# .transform 전까지 : 성별로 나누고 나이 행을 뽑는다
train.groupby('Sex')['Age'].transform('mean')

0      30.726645
1      27.915709
2      27.915709
3      27.915709
4      30.726645
         ...    
886    30.726645
887    27.915709
888    27.915709
889    30.726645
890    30.726645
Name: Age, Length: 891, dtype: float64

In [159]:
# 비어있거나 잘못된 값인 결측치 제거
train['Age'].fillna(train.groupby('Sex')['Age'].transform('mean'), inplace=True)
test['Age'].fillna(test.groupby('Sex')['Age'].transform('mean'), inplace=True)
test['Fare'].fillna(test.groupby('Sex')['Age'].transform('median'), inplace=True)

In [160]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.000000,1,0,7.2500
1,2,1,1,female,38.000000,1,0,71.2833
2,3,1,3,female,26.000000,0,0,7.9250
3,4,1,1,female,35.000000,1,0,53.1000
4,5,0,3,male,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.000000,0,0,13.0000
887,888,1,1,female,19.000000,0,0,30.0000
888,889,0,3,female,27.915709,1,2,23.4500
889,890,1,1,male,26.000000,0,0,30.0000


In [161]:
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [162]:
# 문자열을 숫자로 바꿔주기 위한 매핑
sex_mapping = {"male":0, "female":1}

train['Sex'] = train['Sex'].map(sex_mapping)
test['Sex'] = test['Sex'].map(sex_mapping)

In [163]:
test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,0,34.500000,0,0,7.8292
1,893,3,1,47.000000,1,0,7.0000
2,894,2,0,62.000000,0,0,9.6875
3,895,3,0,27.000000,0,0,8.6625
4,896,3,1,22.000000,1,1,12.2875
...,...,...,...,...,...,...,...
413,1305,3,0,30.272732,0,0,8.0500
414,1306,1,1,39.000000,0,0,108.9000
415,1307,3,0,38.500000,0,0,7.2500
416,1308,3,0,30.272732,0,0,8.0500


In [164]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.000000,1,0,7.2500
1,2,1,1,1,38.000000,1,0,71.2833
2,3,1,3,1,26.000000,0,0,7.9250
3,4,1,1,1,35.000000,1,0,53.1000
4,5,0,3,0,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000
887,888,1,1,1,19.000000,0,0,30.0000
888,889,0,3,1,27.915709,1,2,23.4500
889,890,1,1,0,26.000000,0,0,30.0000


In [165]:
# 왜 나이에 대한 이상치 제거를 하는걸까 ? 
age_mean = train['Age'].mean()
age_std = train['Age'].std()

In [166]:
age_mean

29.736034227171242

In [167]:
age_std

13.01489687938195

In [168]:
indexNames = train[train['Age'] < age_mean - 3*age_std].index

In [169]:
indexNames

Int64Index([], dtype='int64')

In [170]:
indexNames = train[train['Age'] > age_mean + 3*age_std].index

In [171]:
indexNames

Int64Index([96, 116, 493, 630, 672, 745, 851], dtype='int64')

In [172]:
train.drop(indexNames, inplace=True)

In [173]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.000000,1,0,7.2500
1,2,1,1,1,38.000000,1,0,71.2833
2,3,1,3,1,26.000000,0,0,7.9250
3,4,1,1,1,35.000000,1,0,53.1000
4,5,0,3,0,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000
887,888,1,1,1,19.000000,0,0,30.0000
888,889,0,3,1,27.915709,1,2,23.4500
889,890,1,1,0,26.000000,0,0,30.0000


In [174]:
fare_mean = train['Fare'].mean()
fare_std = train['Fare'].std()

print(fare_mean)
print(fare_std)

32.22032341628956
49.85048349003016


In [175]:
indexNames = train[train['Fare'] > fare_mean + 3*fare_std].index

In [176]:
indexNames

Int64Index([ 27,  88, 118, 258, 299, 311, 341, 377, 380, 438, 527, 557, 679,
            689, 700, 716, 730, 737, 742, 779],
           dtype='int64')

In [177]:
train.drop(indexNames, inplace=True)

In [178]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.000000,1,0,7.2500
1,2,1,1,1,38.000000,1,0,71.2833
2,3,1,3,1,26.000000,0,0,7.9250
3,4,1,1,1,35.000000,1,0,53.1000
4,5,0,3,0,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000
887,888,1,1,1,19.000000,0,0,30.0000
888,889,0,3,1,27.915709,1,2,23.4500
889,890,1,1,0,26.000000,0,0,30.0000


In [179]:
from sklearn.linear_model import LogisticRegression

# solver : 알고리즘 종류를 정하는 매개변수로 기본값이 lbfgs 
ml = LogisticRegression(solver='lbfgs')

In [180]:
x = train.drop(['Survived'], axis=1)
y = train['Survived']

In [181]:
ml.fit(x, y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [182]:
ml.coef_

array([[ 2.05498590e-04, -7.42594965e-01,  2.75734210e+00,
        -3.55006293e-02, -4.86365626e-01, -2.48400916e-01,
         1.50801792e-02]])

In [183]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

scoring = 'accuracy'
score = cross_val_score(ml, x, y, cv=k_fold, n_jobs=1, scoring=scoring)

print(score)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[0.77011494 0.7816092  0.73563218 0.79310345 0.84883721 0.75581395
 0.8372093  0.75581395 0.84883721 0.79069767]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [184]:
round(np.mean(score)*100, 2)

79.18

In [187]:
predict = ml.predict(test)

result = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predict
})

result.to_csv('result.csv', index=False)

r = pd.read_csv('result.csv')

r